In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
from tensorflow.keras import layers as kl

In [7]:
tf.__version__

'1.15.0'

In [2]:
# simplest example - gaussian distribution
distr_layer = tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc = t[... , 0], scale = 1))
# zde se tfd.Normal nahradi tfd.NegativeBinomial, u ktereho chci ale odhadovat 2 parametry

In [35]:
# simplest model - linear regression
model = tf.keras.models.Sequential([
    kl.Input([3]),
    kl.Dense(units = 1, use_bias = False),
    distr_layer
    
])
negloglik = lambda x, rv_x: tf.reduce_mean(-rv_x.log_prob(x))
model.compile(optimizer='adam', loss=negloglik)
#model.compile(optimizer='adam', loss='mean_squared_error')
# zde se kl.Dense(units = 1, use_bias = False) nahradi kl.Dense(units = 2, use_bias = False)

In [36]:
N = 100000 # sample size
coefs = np.array([[1, 2, 3]]).T # true linear regression (= dense layer's) coeffs, 3 regressors
#tf.random.set_seed(1)
X = tfd.Normal(1, 1).sample(sample_shape = [N, 3]) # random regression matrix (inputs)
# zde coeffs zmenime na 3 x 2 matici

In [37]:
sess = tf.InteractiveSession()

d:\to_delete_venv\lib\site-packages\tensorflow_core\python\client\session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [38]:
X = sess.run(X)

In [39]:
true_distr_params = X @ coefs # means of the respective normal distributions (Y)
true_distr = distr_layer(true_distr_params)
#tf.random.set_seed(1)
Y = true_distr.sample() # sample some targets

In [40]:
Y = sess.run(Y)

In [41]:
model(X)

<tfp.distributions.Normal 'sequential_5_distribution_lambda_Normal' batch_shape=[100000] event_shape=[] dtype=float32>

In [17]:
np.mean(negloglik(Y, true_distr)) # sample loss with true model

NotImplementedError: Cannot convert a symbolic Tensor (Neg_1:0) to a numpy array.

In [ ]:
np.mean(negloglik(Y, model(X))) # sample loss with randomly initialized model

In [ ]:
model.weights # initial weights

In [42]:
# fit the model (surprisingly does not work!!!)  
# it seems there is some quadratic prior on model parameters and we are not able to turn it off
model.fit(x = X, y = Y, epochs = 100, batch_size=128) 

Train on 100000 samples
Epoch 1/100


InvalidArgumentError: Expected multiples argument to be a vector of length 1 but got length 2
	 [[{{node training_4/Adam/gradients/gradients/loss_3/distribution_lambda_loss/Mean_grad/Tile}}]]

In [ ]:
model.weights # not a very good output

In [ ]:
model.weights[0] * model.weights[1]

In [ ]:
# loss for "manual" optimization
def get_loss():
    return tf.reduce_mean(negloglik(Y, model(X)))

In [ ]:
optimizer = tf.optimizers.Adam() # the same optimizer as above

In [ ]:
# manually apply optimizer's gradient step to the model weights
@tf.function
def train_step():
    with tf.GradientTape() as tape:
        loss = get_loss()

    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss

In [ ]:
# train as long as you wish :-)
for i in range(1000):
    loss = train_step()
    if i % 100 == 0:
        print(loss)

In [ ]:
model.trainable_weights # correct weights